In [ ]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import Planck18 as cosmo
from astropy.coordinates import SkyCoord
from scipy.spatial import cKDTree

### Load catalogs

In [ ]:
bg_cat = Table.read("../data/foreground_catalog.fits")
fg_cat = Table.read("../data/background_catalog.fits")

### Find pairs

In [ ]:
# Find pairs

# Flat-sky approximation
ra = fg_cat["coord_ra"].data
dec = fg_cat["coord_dec"].data
fg_xy = np.column_stack((ra * np.cos(np.deg2rad(dec)), dec))

ra = bg_cat["coord_ra"].data
dec = bg_cat["coord_dec"].data
bg_xy = np.column_stack((ra * np.cos(np.deg2rad(dec)), dec))

# Build spatial KDTree on background
bg_tree = cKDTree(bg_xy)

# Loop over foreground galaxies and find background galaxies within 1 Mpc
pairs = []
impact_params = []
# position_angles = []
for i_fg in range(len(fg_xy)):
    # Calculate angular diameter distance to fg galaxy
    dA = cosmo.angular_diameter_distance(fg_cat["z_phot"][i_fg]).value

    # Calc angle corresponding to 4 Mpc impact parameter at this distance
    max_sep_deg = np.rad2deg(4 / dA)

    # Find matches within this angle
    matches = bg_tree.query_ball_point(fg_xy[i_fg], r=max_sep_deg)

    # Calculate position angles for shear decomposition
    # Note in WL position angle is defined to start from negative RA direction (West) and increase CCW
    # but in Astropy, position angle starts from North. Thus, add pi/2 to Astropy angle
    # fg_coord = SkyCoord(fg_cat["coord_ra"][i_fg], fg_cat["coord_dec"][i_fg], frame="icrs", unit="deg")
    # bg_coord = SkyCoord(bg_cat["coord_ra"], bg_cat["coord_dec"], frame="icrs", unit="deg")
    # pa_all_bg = fg_coord.position_angle(bg_coord).rad + np.pi/2.

    for i_bg in matches:
        # Save pair indices
        pairs.append((i_fg, i_bg))

        # Calculate angular distance
        dx = fg_xy[i_fg, 0] - bg_xy[i_bg, 0]
        dy = fg_xy[i_fg, 1] - bg_xy[i_bg, 1]
        theta = np.deg2rad(np.sqrt(dx**2 + dy**2))

        # Save impact parameter
        impact_params.append(dA * theta)

        # Save position angle
        # position_angles.append(pa_all_bg[i_bg])

pairs = np.array(pairs)
impact_params = np.array(impact_params)
# position_angles = np.array(position_angles)

print(f"Found {len(pairs)/1e6:.2f} million foreground-background pairs.")

### Bin statistics

In [ ]:
# Container for results
avg_flux = {}

# Set bins in impact parameter
bin_edges = np.arange(0.1, 4.4, 0.4)
bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2

# First do every band
for band in "ugrizy":
    print(f"Processing {band} band")

    # Grab fluxes and errors for background galaxies in pairs
    flux = bg_cat[f"{band}_cModelFlux"][pairs[:, 1]]
    err = bg_cat[f"{band}_cModelFluxErr"][pairs[:, 1]]

    # Enforce 1% error floor
    if band == "u":
        err = np.clip(err / flux, 0.05, None) * flux
    # else:
    #    err = np.clip(err / flux, 0.01, None) * flux

    # Weighting and filtering
    mask = np.isfinite(flux) & np.isfinite(err) & (err > 0) & np.isfinite(impact_params)
    flux = flux[mask]
    weights = err[mask] ** -2
    impact = impact_params[mask]

    # Assign bins
    bin_indices = np.digitize(impact, bins=bin_edges) - 1
    in_range = (bin_indices >= 0) & (bin_indices < len(bin_centers))
    bin_indices = bin_indices[in_range]
    flux = flux[in_range]
    weights = weights[in_range]

    # Calculate binned averages
    avgs = []
    errs = []
    for i in range(len(bin_centers)):
        # Find values for this bin
        mask = bin_indices == i

        # If bin is empty, append NaN
        if mask.sum() == 0:
            avgs.append(np.nan)
            errs.append(np.nan)

        # Otherwise calculate average and uncertainty
        else:
            avgs.append(np.average(flux[mask], weights=weights[mask]))

            # Population variance
            var = (
                flux[mask].var() * np.sum(weights[mask] ** 2) / weights[mask].sum() ** 2
            )
            var += 1 / weights[mask].sum()  # Add contribution from measurement errors
            errs.append(np.sqrt(var))

    avg_flux[band] = dict(avg=np.array(avgs), err=np.array(errs))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)

colors = ["C4", "C2", "C3", "C1", "C5", "C7"]
for i, (band, vals) in enumerate(avg_flux.items()):
    # Load fluxes
    avg = vals["avg"].copy()
    err = vals["err"].copy()

    # Normalize to largest impact param
    norm = avg[-1:].mean()
    avg /= norm
    err /= norm

    ax.errorbar(
        bin_centers,
        avg,
        yerr=err,
        label=band,
        marker="o",
        markersize=4,
        capsize=4,
        color=colors[i],
        zorder=10 - i,
    )

ax.legend(frameon=False, ncols=3)
ax.set(
    xlabel="Impact parameter (Mpc)",
    ylabel="Normalized weighted mean flux",
    ylim=(0.998, 1.024),
)
ax.set_title("Fluxes", fontsize=14)

ax.axhline(1.0, c="k", lw=1, ls="--", zorder=0)

fig.savefig("../figures/flux_vs_impact.png", bbox_inches="tight", dpi=500)

In [ ]:
# Container for results
avg_color = {}

# Loop over colors
bands = list("ugrizy")
for i in range(len(bands) - 1):
    band1 = bands[i]
    band2 = bands[i + 1]
    print(f"Processing {band1} - {band2}")

    # Grab fluxes and errors for background galaxies in pairs
    flux1 = bg_cat[f"{band1}_gaap1p0Flux"][pairs[:, 1]]
    err1 = bg_cat[f"{band1}_gaap1p0FluxErr"][pairs[:, 1]]

    flux2 = bg_cat[f"{band2}_gaap1p0Flux"][pairs[:, 1]]
    err2 = bg_cat[f"{band2}_gaap1p0FluxErr"][pairs[:, 1]]

    # Enforce 1% error floor
    if band1 == "u":
        err1 = np.clip(err1 / flux1, 0.05, None) * flux1

    # Calculate color ratio
    color = flux1 / flux2
    err = color * np.sqrt((err1 / flux1) ** 2 + (err2 / flux2) ** 2)

    # Weighting and filtering
    mask = (
        np.isfinite(color) & np.isfinite(err) & (err > 0) & np.isfinite(impact_params)
    )
    if band1 == "i" and band2 == "z":
        fzb = bg_cat["fzboost_z_mode"][pairs[:, 1]]
        fzb_sig = (
            bg_cat["fzboost_z_err68_high"][pairs[:, 1]]
            - bg_cat["fzboost_z_err68_low"][pairs[:, 1]]
        ) / 2
        lph = bg_cat["lephare_z_mode"][pairs[:, 1]]
        lph_sig = (
            bg_cat["lephare_z_err68_high"][pairs[:, 1]]
            - bg_cat["lephare_z_err68_low"][pairs[:, 1]]
        ) / 2
        mask &= (
            (np.abs(fzb - lph) < 0.1 * (1 + bg_cat["z_phot"][pairs[:, 1]]))
            & (fzb_sig < 0.15)
            & (fzb_sig < 0.15)
        )
    color = color[mask]
    weights = err[mask] ** -2
    impact = impact_params[mask]

    # Assign bins
    bin_indices = np.digitize(impact, bins=bin_edges) - 1
    in_range = (bin_indices >= 0) & (bin_indices < len(bin_centers))
    bin_indices = bin_indices[in_range]
    color = color[in_range]
    weights = weights[in_range]

    # Calculate binned averages
    avgs = []
    errs = []
    for i in range(len(bin_centers)):
        # Find values for this bin
        mask = bin_indices == i

        # If bin is empty, append NaN
        if mask.sum() == 0:
            avgs.append(np.nan)
            errs.append(np.nan)

        # Otherwise calculate average and uncertainty
        else:
            avgs.append(np.average(color[mask], weights=weights[mask]))

            # Population variance
            var = (
                color[mask].var()
                * np.sum(weights[mask] ** 2)
                / weights[mask].sum() ** 2
            )
            var += 1 / weights[mask].sum()  # Add contribution from measurement errors
            errs.append(np.sqrt(var))

    avg_color[f"{band1}-{band2}"] = dict(avg=np.array(avgs), err=np.array(errs))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)

colors = ["C4", "C2", "C3", "C1", "C5", "C7"]
for i, (color, vals) in enumerate(avg_color.items()):
    # Error is way too large
    if color == "z-y":
        continue

    # Load fluxes
    avg = vals["avg"].copy()
    err = vals["err"].copy()

    # Convert to magnitudes
    avg = -2.5 * np.log10(avg)
    err = np.abs(2.5 * np.log10(1 + err / avg))

    # Normalize to largest impact param
    norm = avg[-1:].mean()
    avg /= norm
    err /= norm

    ax.errorbar(
        bin_centers,
        avg,
        yerr=err,
        label=color,
        marker="o",
        markersize=4,
        capsize=4,
        color=colors[i],
    )

ax.legend(frameon=False, ncols=2)
ax.set(
    xlabel="Impact parameter (Mpc)",
    ylabel="Normalized weighted mean color",
    ylim=(0.9975, 1.022),
)
ax.set_title("Colors", fontsize=14)

ax.axhline(1.0, c="k", lw=1, ls="--", zorder=0)
fig.savefig("../figures/color_vs_impact.png", bbox_inches="tight", dpi=500)

In [ ]:
# Container for results
avg_shear_t = []
err_shear_t = []
avg_shear_x = []
err_shear_x = []

# Grab shears and errors for background galaxies in pairs
g1 = bg_cat["shear_g1"][pairs[:, 1]]
g2 = bg_cat["shear_g2"][pairs[:, 1]]
err = bg_cat["shear_err"][pairs[:, 1]]
use_shear = bg_cat["use_shear"][pairs[:, 1]]

# Weighting and filtering
mask = (
    np.isfinite(g1)
    & np.isfinite(g2)
    & np.isfinite(err)
    & (err > 0)
    & use_shear
    & np.isfinite(impact_params)
)
g1 = g1[mask]
g2 = g2[mask]
weights = err[mask] ** -2
impact = impact_params[mask]
pa = position_angles[mask]

# Calculate tangential and cross shear
gt = -g1 * np.cos(2 * pa) - g2 * np.sin(2 * pa)
gx = g1 * np.sin(2 * pa) - g2 * np.cos(2 * pa)

# Assign bins
bin_indices = np.digitize(impact, bins=bin_edges) - 1
in_range = (bin_indices >= 0) & (bin_indices < len(bin_centers))
bin_indices = bin_indices[in_range]
gt = gt[in_range]
gx = gx[in_range]
weights = weights[in_range]

# Calculate binned averages
avgs = []
errs = []
for i in range(len(bin_centers)):
    # Find values for this bin
    mask = bin_indices == i

    # If bin is empty, append NaN
    if mask.sum() == 0:
        avgs.append(np.nan)
        errs.append(np.nan)

    # Otherwise calculate average and uncertainty
    else:
        # First tangential shear
        avg_shear_t.append(np.average(gt[mask], weights=weights[mask]))
        # Population variance
        var = gt[mask].var() * np.sum(weights[mask] ** 2) / weights[mask].sum() ** 2
        var += 1 / weights[mask].sum()  # Add contribution from measurement errors
        err_shear_t.append(np.sqrt(var))

        # Now cross shear
        avg_shear_x.append(np.average(gx[mask], weights=weights[mask]))
        var = gx[mask].var() * np.sum(weights[mask] ** 2) / weights[mask].sum() ** 2
        var += 1 / weights[mask].sum()  # Add contribution from measurement errors
        err_shear_x.append(np.sqrt(var))

avg_shear_t = np.array(avg_shear_t)
err_shear_t = np.array(err_shear_t)
avg_shear_x = np.array(avg_shear_x)
err_shear_x = np.array(err_shear_x)

In [ ]:
fig, ax = plt.subplots()

# Normalize to largest impact param
norm = avg[np.isfinite(avg)][-1]
avg /= norm
err /= norm

ax.errorbar(
    bin_centers,
    avg_shear_t,
    yerr=err_shear_t,
    marker="o",
    capsize=4,
    label="$g_\\text{T}$",
)

ax.errorbar(
    bin_centers,
    avg_shear_x,
    yerr=err_shear_x,
    marker="o",
    capsize=4,
    label="$g_\\text{X}$",
)

ax.legend(frameon=False, ncols=2)
ax.set(
    xlabel="Impact parameter (Mpc)",
    ylabel="Shear",
)

fig.savefig("../figures/shear_vs_impact.png", bbox_inches="tight", dpi=500)